In [1]:
import os, re, glob, json
from pathlib import Path
import numpy as np
import pandas as pd
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.second_level import SecondLevelModel, non_parametric_inference
from nilearn import image
import matplotlib.pyplot as plt
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_stat_map
from nilearn.glm import threshold_stats_img
from nilearn import plotting
from nilearn.image import load_img, resample_to_img

In [2]:
BASE = "/local/anpa/ds003242-1.0.0"
DERIVATIVES = f"{BASE}/derivatives/"
DERIVATIVES_FMRIPREP = f"{DERIVATIVES}/fmriprep/" 
DERIV_OUT = f"{DERIVATIVES}/firstlevel_separate_runs/"  
TASK = "CIC"
TR = 2.0

In [3]:
NUIS = ["framewise_displacement",
        "a_comp_cor_00","a_comp_cor_01","a_comp_cor_02",
        "a_comp_cor_03","a_comp_cor_04","a_comp_cor_05"]

In [4]:
def fprep_func_dir(sub):
    return Path(BASE)/f"derivatives/fmriprep/sub-{sub}/func"


def collect_runs(sub, task):
    func = fprep_func_dir(sub)
    patt = func.glob(f"sub-{sub}_task-{task}_run-*_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz")
    runs = []
    for p in sorted(patt):
        m = re.search(r"_run-(\d+)_", p.name)
        if m: runs.append(m.group(1))
    return runs


def files_for(sub, task, run):
    func = fprep_func_dir(sub)
    bold = func/f"sub-{sub}_task-{task}_run-{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
    conf = func/f"sub-{sub}_task-{task}_run-{run}_desc-confounds_regressors.tsv"
    evt  = Path(BASE)/f"sub-{sub}/func/sub-{sub}_task-{task}_run-{run}_events.tsv"
    return bold, conf, evt


def load_confounds_list(conf_paths):
    out=[]
    for cp in conf_paths:
        df = pd.read_csv(cp, sep="\t").fillna(0)
        for c in NUIS:
            if c not in df: df[c]=0.0
        out.append(df[NUIS])
    return out


In [5]:
def resample_mask_to_bold(anat_mask, bold_img):
    """Resample anat mask to the space of the bold image.
    As in tutorial https://nilearn.github.io/dev/auto_examples/04_glm_first_level/plot_first_level_details.html
    """
    return resample_to_img(
        anat_mask,
        bold_img,
        interpolation="nearest",
        copy_header=True,
        force_resample=True,
    )

In [6]:
def get_runs_files(sub, task=TASK):
    """Collect all runs for a subject and task."""
    runs = collect_runs(sub, task)
    bolds, confs, evts = [], [], []
    for run in runs:
        b, c, e = files_for(sub, task, run)
        if b.exists() and c.exists() and e.exists():
            bolds.append(str(b))
            confs.append(str(c))
            evts.append(pd.read_csv(e, sep="\t"))
    confs= load_confounds_list(confs)
    anat_mask  = f'{DERIVATIVES_FMRIPREP}/sub-{sub}/anat/sub-{sub}_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz'
    
    resampled_anat_mask = resample_mask_to_bold(
        load_img(anat_mask),
        load_img(bolds[0])
    ) 
    return bolds, confs, evts, resampled_anat_mask

In [7]:
all_sub_dirs = sorted([p.name.split("sub-")[-1] for p in Path(DERIVATIVES_FMRIPREP).glob("sub-*") if p.is_dir()])
all_sub_dirs[:3]

['SAXSISO01b', 'SAXSISO01f', 'SAXSISO01s']

In [ ]:
conditions = ['Food_1', 'Food_2', 'Food_3',
              'Social_1', 'Social_2', 'Social_3',
              'Control_1', 'Control_2', 'Control_3']

# Map fine-grained regressors to broader class labels
label_map = {
    'Food_1': 'Food',
    'Food_2': 'Food',
    'Food_3': 'Food',
    'Social_1': 'Social',
    'Social_2': 'Social',
    'Social_3': 'Social',
    'Control_1': 'Control',
    'Control_2': 'Control',
    'Control_3': 'Control'
}

for sub_sess in all_sub_dirs:
    outdir = Path(DERIV_OUT)/f"sub-{sub_sess}"
    if not outdir.exists(): 
        outdir.mkdir()
        print(f"Processing {sub_sess}")
        bolds, confs, evts, resampled_anat_mask = get_runs_files(sub_sess, TASK)
        for run_id in range(len(bolds)): 
            glm = FirstLevelModel(
                t_r=TR, hrf_model="spm", drift_model="cosine", high_pass=0.008,
                noise_model="ar1", smoothing_fwhm=2, standardize=True,
                mask_img=resampled_anat_mask, minimize_memory=True, n_jobs=4
            ).fit(run_imgs=bolds[run_id], events=evts[run_id], confounds=confs[run_id])

            for cond in conditions:
                eff = glm.compute_contrast(cond, output_type='all', stat_type='t')
                eff['effect_size'].to_filename(outdir / f"{run_id}_{cond}_effsize.nii.gz")
                eff['effect_variance'].to_filename(outdir / f"{run_id}_{cond}_effvar.nii.gz")
                eff['z_score'].to_filename(outdir / f"{run_id}_{cond}_zmap.nii.gz")
                eff['stat'].to_filename(outdir / f"{run_id}_{cond}_tstat.nii.gz")



Processing SAXSISO01b


/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: Use

Processing SAXSISO32b


/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: Use

Processing SAXSISO32f


...

/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: Use

Processing SAXSISO40b


/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: Use

Processing SAXSISO40f


/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: The following unexpected columns in events data will be ignored: weight
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: UserWarning: [NiftiMasker.fit] Generation of a mask has been requested (imgs != None) while a mask was given at masker creation. Given mask will be used.
  glm = FirstLevelModel(
/tmp/ipykernel_258484/1860583228.py:27: Use